# AES

In [37]:
import numpy as np

In [42]:
def get_state(msg):
    if len(msg) != 16:
        return -1
    
    state = msg
    state = [ord(char) for char in state]
    state = [hex(char) for char in state]
    state = [int(char, 16) for char in state]
    state = [state[i:i+4] for i in range(0, len(state), 4)]
    return np.array(state)

def get_msg(state):
    msg = state.copy()
    msg = [char for row in state for char in row]
    msg = ''.join(chr(char) for char in msg)
    return msg

def print_hex(state):
    for row in state:
        for char in row:
            print(hex(char), end=' ')
        print()

## Substitute Bytes


In [4]:
s_box = np.array([
    [0x63,	0x7c,	0x77,	0x7b,	0xf2,	0x6b,	0x6f,	0xc5,	0x30,	0x01,	0x67,	0x2b,	0xfe,	0xd7,	0xab,	0x76],
    [0xca,	0x82,	0xc9,	0x7d,	0xfa,	0x59,	0x47,	0xf0,	0xad,	0xd4,	0xa2,	0xaf,	0x9c,	0xa4,	0x72,	0xc0],
    [0xb7,	0xfd,	0x93,	0x26,	0x36,	0x3f,	0xf7,	0xcc,	0x34,	0xa5,	0xe5,	0xf1,	0x71,	0xd8,	0x31,	0x15],
    [0x04,	0xc7,	0x23,	0xc3,	0x18,	0x96,	0x05,	0x9a,	0x07,	0x12,	0x80,	0xe2,	0xeb,	0x27,	0xb2,	0x75],
    [0x09,	0x83,	0x2c,	0x1a,	0x1b,	0x6e,	0x5a,	0xa0,	0x52,	0x3b,	0xd6,	0xb3,	0x29,	0xe3,	0x2f,	0x84],
    [0x53,	0xd1,	0x00,	0xed,	0x20,	0xfc,	0xb1,	0x5b,	0x6a,	0xcb,	0xbe,	0x39,	0x4a,	0x4c,	0x58,	0xcf],
    [0xd0,	0xef,	0xaa,	0xfb,	0x43,	0x4d,	0x33,	0x85,	0x45,	0xf9,	0x02,	0x7f,	0x50,	0x3c,	0x9f,	0xa8],
    [0x51,	0xa3,	0x40,	0x8f,	0x92,	0x9d,	0x38,	0xf5,	0xbc,	0xb6,	0xda,	0x21,	0x10,	0xff,	0xf3,	0xd2],
    [0xcd,	0x0c,	0x13,	0xec,	0x5f,	0x97,	0x44,	0x17,	0xc4,	0xa7,	0x7e,	0x3d,	0x64,	0x5d,	0x19,	0x73],
    [0x60,	0x81,	0x4f,	0xdc,	0x22,	0x2a,	0x90,	0x88,	0x46,	0xee,	0xb8,	0x14,	0xde,	0x5e,	0x0b,	0xdb],
    [0xe0,	0x32,	0x3a,	0x0a,	0x49,	0x06,	0x24,	0x5c,	0xc2,	0xd3,	0xac,	0x62,	0x91,	0x95,	0xe4,	0x79],
    [0xe7,	0xc8,	0x37,	0x6d,	0x8d,	0xd5,	0x4e,	0xa9,	0x6c,	0x56,	0xf4,	0xea,	0x65,	0x7a,	0xae,	0x08],
    [0xba,	0x78,	0x25,	0x2e,	0x1c,	0xa6,	0xb4,	0xc6,	0xe8,	0xdd,	0x74,	0x1f,	0x4b,	0xbd,	0x8b,	0x8a],
    [0x70,	0x3e,	0xb5,	0x66,	0x48,	0x03,	0xf6,	0x0e,	0x61,	0x35,	0x57,	0xb9,	0x86,	0xc1,	0x1d,	0x9e],
    [0xe1,	0xf8,	0x98,	0x11,	0x69,	0xd9,	0x8e,	0x94,	0x9b,	0x1e,	0x87,	0xe9,	0xce,	0x55,	0x28,	0xdf],
    [0x8c,	0xa1,	0x89,	0x0d,	0xbf,	0xe6,	0x42,	0x68,	0x41,	0x99,	0x2d,	0x0f,	0xb0,	0x54,	0xbb,	0x16],
])

inverse_s_box = np.array([
    [0x52,	0x09,	0x6a,	0xd5,	0x30,	0x36,	0xa5,	0x38,	0xbf,	0x40,	0xa3,	0x9e,	0x81,	0xf3,	0xd7,	0xfb],
    [0x7c,	0xe3,	0x39,	0x82,	0x9b,	0x2f,	0xff,	0x87,	0x34,	0x8e,	0x43,	0x44,	0xc4,	0xde,	0xe9,	0xcb],
    [0x54,	0x7b,	0x94,	0x32,	0xa6,	0xc2,	0x23,	0x3d,	0xee,	0x4c,	0x95,	0x0b,	0x42,	0xfa,	0xc3,	0x4e],
    [0x08,	0x2e,	0xa1,	0x66,	0x28,	0xd9,	0x24,	0xb2,	0x76,	0x5b,	0xa2,	0x49,	0x6d,	0x8b,	0xd1,	0x25],
    [0x72,	0xf8,	0xf6,	0x64,	0x86,	0x68,	0x98,	0x16,	0xd4,	0xa4,	0x5c,	0xcc,	0x5d,	0x65,	0xb6,	0x92],
    [0x6c,	0x70,	0x48,	0x50,	0xfd,	0xed,	0xb9,	0xda,	0x5e,	0x15,	0x46,	0x57,	0xa7,	0x8d,	0x9d,	0x84],
    [0x90,	0xd8,	0xab,	0x00,	0x8c,	0xbc,	0xd3,	0x0a,	0xf7,	0xe4,	0x58,	0x05,	0xb8,	0xb3,	0x45,	0x06],
    [0xd0,	0x2c,	0x1e,	0x8f,	0xca,	0x3f,	0x0f,	0x02,	0xc1,	0xaf,	0xbd,	0x03,	0x01,	0x13,	0x8a,	0x6b],
    [0x3a,	0x91,	0x11,	0x41,	0x4f,	0x67,	0xdc,	0xea,	0x97,	0xf2,	0xcf,	0xce,	0xf0,	0xb4,	0xe6,	0x73],
    [0x96,	0xac,	0x74,	0x22,	0xe7,	0xad,	0x35,	0x85,	0xe2,	0xf9,	0x37,	0xe8,	0x1c,	0x75,	0xdf,	0x6e],
    [0x47,	0xf1,	0x1a,	0x71,	0x1d,	0x29,	0xc5,	0x89,	0x6f,	0xb7,	0x62,	0x0e,	0xaa,	0x18,	0xbe,	0x1b],
    [0xfc,	0x56,	0x3e,	0x4b,	0xc6,	0xd2,	0x79,	0x20,	0x9a,	0xdb,	0xc0,	0xfe,	0x78,	0xcd,	0x5a,	0xf4],
    [0x1f,	0xdd,	0xa8,	0x33,	0x88,	0x07,	0xc7,	0x31,	0xb1,	0x12,	0x10,	0x59,	0x27,	0x80,	0xec,	0x5f],
    [0x60,	0x51,	0x7f,	0xa9,	0x19,	0xb5,	0x4a,	0x0d,	0x2d,	0xe5,	0x7a,	0x9f,	0x93,	0xc9,	0x9c,	0xef],
    [0xa0,	0xe0,	0x3b,	0x4d,	0xae,	0x2a,	0xf5,	0xb0,	0xc8,	0xeb,	0xbb,	0x3c,	0x83,	0x53,	0x99,	0x61],
    [0x17,	0x2b,	0x04,	0x7e,	0xba,	0x77,	0xd6,	0x26,	0xe1,	0x69,	0x14,	0x63,	0x55,	0x21,	0x0c,	0x7d],
])

In [46]:
def substitute_bytes(state):
    res = state.copy()
    for i in range(4):
        for j in range(4):
            res[i][j] = s_box[res[i][j] // 16][res[i][j] % 16]
    return res

def inverse_substitute_bytes(state):
    res = state.copy()
    for i in range(4):
        for j in range(4):
            res[i][j] = inverse_s_box[res[i][j] // 16][res[i][j] % 16]
    return res

## Shift Rows


In [47]:
def shift_rows(state):
    res = state.copy()
    for i in range(4):
        for j in range(4):
            res[i][j] = state[i][(j+i) % 4]
    return res

def inverse_shift_rows(state):
    res = state.copy()
    for i in range(4):
        for j in range(4):
            res[i][j] = state[i][(j-i) % 4]
    return res

## Mix Columns


In [48]:
mix_col_matrix = np.array([
    [2, 3, 1, 1],
    [1, 2, 3, 1],
    [1, 1, 2, 3],
    [3, 1, 1, 2],
])

inverse_mix_col_matrix = np.array([
    [14, 11, 13,  9],
    [ 9, 14, 11, 13],
    [13,  9, 14, 11],
    [11, 13,  9, 14]
])

In [50]:
def gmul(x, y):
    p = 0b100011011             # irreducable polynomial x^8+x^4+x^3+x+1
    m = 0                       # m will be product
    for i in range(8):          # mul two 8 bit values
        m = m << 1
        if m & 0b100000000:     # m overflow
            m = m ^ p           # reduce m by p using XOR
        if y & 0b010000000:     # if y is 1
            m = m ^ x           # add x to m using XOR
        y = y << 1
    return m

def mix_columns(state, matrix):
    res = state.copy()
    for i in range(4):
        for j in range(4):
            res[i][j] = 0
            for k in range(4):
                res[i][j] ^= gmul(matrix[i][k], state[k][j])
    return res

## Add Round Key

In [51]:
def add_round_key(state, round_key):
    res = state.copy()
    for i in range(4):
        for j in range(4):
            res[i][j] ^= round_key[i][j]
    return res

# Key Expansion

In [52]:
round_constant = np.array([
    [0x01, 0x00, 0x00, 0x00],
    [0x02, 0x00, 0x00, 0x00],
    [0x04, 0x00, 0x00, 0x00],
    [0x08, 0x00, 0x00, 0x00],
    [0x10, 0x00, 0x00, 0x00],
    [0x20, 0x00, 0x00, 0x00],
    [0x40, 0x00, 0x00, 0x00],
    [0x80, 0x00, 0x00, 0x00],
    [0x1b, 0x00, 0x00, 0x00],
    [0x36, 0x00, 0x00, 0x00],
])

In [53]:
def rot_word(word):
    res = word.copy()
    for i in range(4):
        res[i] = word[(i+1) % 4]
    return res

def sub_word(word):
    return [s_box[b // 16][b % 16] for b in word]

def key_expansion(key):
    key_symbols = get_state(key).transpose()
    key_schedule = [[] for i in range(44)]

    for i in range(4):
        key_schedule[i] = key_symbols[i]

    for i in range(4, 44):
        temp = key_schedule[i-1]
        if i % 4 == 0:
            temp = sub_word(rot_word(temp)) ^ round_constant[i//4 - 1]
        key_schedule[i] = key_schedule[i-4] ^ temp
        
    key_schedule = np.array(key_schedule).transpose()
    
    res = np.array([key_schedule[:, i:i+4] for i in range(0, 44, 4)])
    
    return res

# AES

In [34]:
def encrypt_aes(msg, key):
    round_keys = key_expansion(key)
    
    state = get_state(msg)
    state = add_round_key(state, round_keys[0])
    
    for i in range(1, 10):
        state = substitute_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state, mix_col_matrix)
        state = add_round_key(state, round_keys[i])
        
    state = substitute_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, round_keys[10])
    
    return get_msg(state)

def decrypt_aes(msg, key):
    round_keys = key_expansion(key)
    
    state = get_state(msg)
    state = add_round_key(state, round_keys[10])
    
    for i in range(9, 0, -1):
        state = inverse_shift_rows(state)
        state = inverse_substitute_bytes(state)
        state = add_round_key(state, round_keys[i])
        state = mix_columns(state, inverse_mix_col_matrix)
        
    state = inverse_shift_rows(state)
    state = inverse_substitute_bytes(state)
    state = add_round_key(state, round_keys[0])
    
    return get_msg(state)

In [54]:
msg = 'Two One Nine Two'
key = 'Thats my Kung Fu'

chiper = encrypt_aes(msg, key)
plain = decrypt_aes(chiper, key)

print(chiper)
print(plain)

àfßó Öäêfýqä
Two One Nine Two


In [ ]:
AES_TYPE = {
    'AES-128': ....
    ...
    ...
}